# nbd

> Extra command line utilities for nbdev

In [ ]:
#| default_exp nbd

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.all import *
import itertools as it
import os
from ghapi.all import *

import configparser
from pathlib import Path

from oztools.core import *
from oztools.gh import *
from nbdev.cli import *
from nbdev.clean import *
from nbdev.quarto import *

import subprocess

In [ ]:
#| export
@call_parse
def nbd_new(name:str, description:str, license:str="Apache-2.0", private:bool=False):
    repo, cloned = gh_new_repo_fn(name, description, license, private)
    os.chdir(repo.name)
    subprocess.run(["nbdev_new"])
    subprocess.run(["nbdev_install_hooks"])
    subprocess.run(["nbdev_prepare"])
    subprocess.run(["nbdev_clean"])
    commit_and_push(cloned, "Initial commit")
    #nbdev_new()
    #nbdev_prepare()
    #nbdev_clean()

In [ ]:
#repo, cloned = gh_new_repo_fn("foo6", "I am not spamming", "Apache-2.0", False)

In [ ]:
nbdev_install_hooks??

Signature: nbdev_install_hooks()
Source:   
@call_parse
def nbdev_install_hooks():
    "Install Jupyter and git hooks to automatically clean, trust, and fix merge conflicts in notebooks"
    cfg_path = Path.home()/'.jupyter'
    cfg_path.mkdir(exist_ok=True)
    cfg_fns = [cfg_path/f'jupyter_{o}_config.py' for o in ('notebook','server')]
    for fn in cfg_fns:
        src = fn.read_text() if fn.exists() else ''
        upd = _add_jupyter_hooks(src, fn)
        if upd is not None: fn.write_text(upd)

    repo_path = _git_root()
    if repo_path is None:
        sys.stderr.write('Not in a git repository, git hooks cannot be installed.\n')
        return
    hook_path = repo_path/'.git'/'hooks'
    fn = hook_path/'post-merge'
    hook_path.mkdir(parents=True, exist_ok=True)
    fn.write_text("#!/bin/bash\nnbdev_trust")
    os.chmod(fn, os.stat(fn).st_mode | stat.S_IEXEC)

    cmd = 'git config --local include.path ../.gitconfig'
    (repo_path/'.gitconfig').write_text(f'''# Generated by n

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()